In [6]:
import pandas as pd
from bs4 import BeautifulSoup
from sqlalchemy import create_engine
import re
import pymysql
import requests

In [2]:
temp_df = pd.read_html(url")[0]
stock = temp_df[["회사명", "종목코드"]]

In [80]:
page_nums = [str(n) for n in range(1, 4)]

In [87]:
def process_df(d):
    engine = create_engine("mysql+pymysql://name:"+"pw"+"@ip:port/엔티티명?charset=utf8", encoding='utf8')
    conn = engine.connect()
    use_df = pd.DataFrame(d)
#     d['date']=pd.to_datetime(d['date'])
    use_df.to_sql(name='tableanme', con=engine, if_exists='append', index=False)
    conn.close()
    del d

In [3]:
headers = {"User-Agent":"Mozilla/5.0 "}


temp_df = pd.DataFrame()

for i in stock.values[]:
    urls = "url"
    code = str(i[1])
    last_page = False
    diff_num = 0
    diff_temp = []
    temp_dict={}
    temp_dict['date'] = []
    temp_dict['close'] = []
    temp_dict['diff'] = []
    temp_dict['start'] = []
    temp_dict['high'] = []
    temp_dict['low'] = []
    temp_dict['volume'] = []
    temp_dict['stock_code'] = []
    temp_dict['stock_name'] = []
    
    if len(code) < 6:
        code = code.zfill(6)
    urls += code
    
    for page_num in page_nums:
        response = requests.get(urls+"&page="+str(page_num), headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        table = soup.find("table")
        trs = table.find_all('tr')
        for tr in trs:
            if len(tr.find_all('td')) > 3:
                if tr.find_all('td')[0].text in temp_dict['date']:
                    last_page = True
                    break
                diff_str = tr.find_all('td')[2]
                if str(type(diff_str.find('span'))) == "<class 'NoneType'>":
                    last_page = True
                    break
                temp_dict['date'].append(re.sub("\.","-",tr.find_all('td')[0].text)) # 날짜
                temp_dict['close'].append((re.sub(",*","",tr.find_all('td')[1].text))) # 종가 
                
                diff_num = int(re.sub(",*","",diff_str.find('span').text).split()[0]) # 전일비 2
                
                if "nv01" in diff_str.find('span')['class']:
                    temp_dict['diff'].append(diff_num*-1)
                else :
                    temp_dict['diff'].append(diff_num)
                temp_dict['start'].append((re.sub(",*","",tr.find_all('td')[3].text))) # 시가
                temp_dict['high'].append((re.sub(",*","",tr.find_all('td')[4].text))) # 고가
                temp_dict['low'].append((re.sub(",*","",tr.find_all('td')[5].text))) # 종가 
                temp_dict['volume'].append((re.sub(",*","",tr.find_all('td')[6].text))) # 거래량 
                temp_dict['stock_code'].append(str(code))
                temp_dict['stock_name'].append(i[0])
        if last_page == True:
            last_page = False
            break
    process_df(temp_dict)
    del temp_dict
    
print("종료")
    

종료


In [77]:
db = pymysql.connect(host='ip주소', port='port number', user='username', passwd='pw', db='table_name', charset='utf8')

In [1]:
sql = "select * from stock"
cursor = db.cursor(pymysql.cursors.DictCursor)
cursor.execute(sql)
resultset = cursor.fetchall()
for result in resultset:
    print(result)

In [ ]:
db.close()